# Testing Tracery

From a GitHub gist [here](https://gist.github.com/aparrish/73c19a36b9cdcf604d04e95020418cd4)


Note: a great how-to guide [here](https://github.com/BenziAhamed/Tracery)

# Resources

- [Plaintext lists](https://www.plaintextlist.com) - not sure what usage but seems useable
- [Wordnet](https://wordnet.princeton.edu) - for dictionary, thesaurus
- [Squinky](https://www.groundai.com/project/squinky-a-corpus-of-sentence-level-formality-informativeness-and-implicature/2) - corpus of sentence-level formality. What is this? Will look later.

In [3]:
!pip3 install tracery

In [13]:
import tracery
from tracery.modifiers import base_english

rules = {
    "origin": "#hello.capitalize#, #location#!",
    "hello": ["hello", "greetings", "howdy", "hey"],
    "location": ["world", "solar system", "galaxy", "universe"]
}

grammar = tracery.Grammar(rules) # create a grammar object from the rules
grammar.add_modifiers(base_english) # add pre-programmed modifiers

for i in range(30):
    print (grammar.flatten("#origin#")) # and flatten, starting with origin rule


Howdy, universe!
Hey, universe!
Greetings, world!
Hello, solar system!
Hey, galaxy!
Hello, world!
Greetings, world!
Hey, universe!
Greetings, world!
Hello, solar system!
Hello, universe!
Hey, world!
Greetings, universe!
Hey, universe!
Hey, world!
Hey, solar system!
Hey, world!
Hello, solar system!
Greetings, galaxy!
Hello, world!
Greetings, universe!
Greetings, universe!
Greetings, world!
Hey, galaxy!
Howdy, galaxy!
Hello, solar system!
Greetings, world!
Hello, galaxy!
Howdy, galaxy!
Hey, world!


What do I want from the generator?

The main thing works as follows:


- Define 2-5 local towns (e.g. Alphatown, Betatown, Charlietown, Deltatown) with one as main town (this becomes front page of directory)

- Generate types of business (in alphabetical order?)
    - Is this subject-level generation? e.g. aardvark, aardvark clearning, aardvark services, aardvark, etc? Or Aardvark, Betamax, etc?
    - How do we relate the genre of business to the businesses generated? Does the type of business become a tracery grammar?  
- Generate individual businesses.
    - Layout type. Options: 
        - Long / short
        - if long then number 0-10 (this becomes layout no, as in one two three - which then determines layout number loaded in static site generator).
        - if short then determine number between 1-100 to make X number of short entries; If short, then only generate name/address/phone no
    - e.g. within Aardvarks, Mick's Aardvark Company, Aardvark Incorporated, _thesaurus word for aardvark_ Services Inc, etc
    - Plus address:
        - Number (0-9999)
        - Street, Street type
        - Town - if in NOT MAIN town then perhaps the slogan becomes more competitive (e.g. "Don't pay TOWN A prices for your Aardvarks!", "Why go to the big city when you want Aardvarks? THE TOWN has all you need.", etc etc.
        - Zip code (define starting two digits for area, if TOWN A then next two digits defined? - how do US Zip codes work?)
    - Plus slogan:
        - FIRST WORD (e.g. Mick) has (it!) / (you (covered/sorted/etc)) / ((what/all/everything) you need / etc.) OR (BUSINESS_TYPE large and small / 
        - or random_adjective pl.BUSINESS_TYPE! random_adjective pl.BUSINESS_TYPE! random_adjective pl.BUSINESS_TYPE! (e.g. Red Aardvarks! Large Aardvarks! Triangular Aardvarks!)
        - Phone no. - AREA_CODE (determined by town), -###-####
    - Plus random number, 0-10000.png * 2 (one for bgd, one for foreground)
    
    - For each part of the info, check against database.db for that category - if it's already an entry, regenerate. If entire data is generated, save into DB.
    
    
    
These are saved as a MarkDown file:

- title (BUSINESS_NAME)
- layout (small / one/two/three/four/five)
- category (e.g. aardvarks)

and as an entry in the database.



Example of a small entries in the ELECTRICIAN category:

**F MASON & SONS LTD** 132 Heath St, Edgemonton. ☎ 434-7737  
**GERRY HEART ELECTRICIAN*** 552 Oakdale Ave, Beaverton. ☎ 434-9310


Which would be rendered in MarkDown as a file named f_mason_sons.md

```
---
layout: small
category: electrician
name: F. Mason & Sons Ltd
address: 132 Heath St, Edgemonton
phone: 434-7737
---
```

-----

So, let's go through those things in order:

# Town names

Let's just start with a phonetic alphabet, then generic town names like -ton, -chester, etc. These are loaded into the files ```corpora/townprefixes.txt``` and ```corpora/townsuffixes.txt```

In [159]:
with open('corpora/townprefixes.txt','r') as f:
    townprefixes = [line.strip() for line in f]
with open('corpora/townsuffixes.txt','r') as f:
    townsuffixes = [line.strip() for line in f]

print("prefixes = ",townprefixes)
print("suffixes =",townsuffixes)

# Let's test this

rules = {
    "origin": "#prefix.capitalize##suffix#",
    "prefix": townprefixes,
    "suffix": townsuffixes
}

grammar = tracery.Grammar(rules) # create a grammar object from the rules
grammar.add_modifiers(base_english) # add pre-programmed modifiers

for i in range(30):
    print (grammar.flatten("#origin#")) # and flatten, starting with origin rule

prefixes =  ['Alfa', 'Bravo', 'Charlie', 'Delta', 'Echo', 'Foxtrot', 'Golf', 'Hotel', 'India', 'Juliett', 'Kilo', 'Lima', 'Mike', 'November', 'Oscar', 'Papa', 'Quebec', 'Romeo', 'Sierra', 'Tango', 'Uniform', 'Victor', 'Whiskey', 'Xray', 'Yankee', 'Zulu']
suffixes = ['ville', 'town', 'ton', 'dam', 'ster', 'stone', 'ston', 'field', 'view', 'son', 'land', 'ford', 'bridge', 'port', 'burn', 'chester', 'side']
Novemberview
Bravoview
Limastone
Uniformside
Charliebridge
Victorland
Quebecfield
Sierraton
Papaston
Papaford
Deltafield
Kiloburn
Kiloson
Quebecside
Papaston
Kilofield
Uniformford
Alfaford
Mikeview
Romeoburn
Tangodam
Deltaside
Indiatown
Novemberport
Tangodam
Foxtrottown
Deltatown
Oscarport
Victorford
Victorburn


In [149]:
# Addresses

def loadData():
    with open('corpora/townprefixes.txt','r') as f:
        townprefixes = [line.strip() for line in f]
    with open('corpora/townsuffixes.txt','r') as f:
        townsuffixes = [line.strip() for line in f]
    with open('corpora/monopolystreets.txt','r') as f:
        street = [line.strip() for line in f]
#     with open('corpora/words_alpha.txt','r') as f:
#         words = [line.strip() for line in f]

    # print("prefixes = ",townprefixes)
    # print("suffixes =",townsuffixes)

loadData()

rules = {
    "address": "#number# #street.capitalize#, #town.capitalize#, #zipcode#",
    "number": ["#d0#","#d0##d#","#d0##d##d#","#d0##d##d#","#d0##d##d#","#d0##d##d#","#d0##d##d##d#","#d0##d##d##d#","#d0##d##d##d#","#d0##d##d##d#","#d0##d##d##d#","#d0##d##d##d#","#d0##d##d##d#","#d0##d##d##d#","#d0##d##d##d#"],
    "street": street,
    "town": "#prefix.capitalize##suffix#",
    "prefix": townprefixes,
    "suffix": townsuffixes,
    "zipcode": "#d##d##d##d##d#",
    "d":["0","1","2","3","4","5","6","7","8","9"],
    "d0":["1","2","3","4","5","6","7","8","9"],
    "words": words
}

grammar = tracery.Grammar(rules) # create a grammar object from the rules
grammar.add_modifiers(base_english) # add pre-programmed modifiers

for i in range(30):
    print (grammar.flatten("#address#")) # and flatten, starting with origin rule

IndexError: Cannot choose from an empty sequence

# zipcodes

Note that US zipcodes are v specific:

    - first digit is state (several states share each number)
    - second + third digits = sectional sorting code
    - fourth + fifth are specific towns, streets, etc
    
...but there are unused numbers! [See here](https://en.wikipedia.org/wiki/Sectional_center_facility)

So later on in the project, let's find ones that don't exist and use only those. That will probably locate us within a state or two too.


In [58]:
# NOTE THIS CELL DOESN'T WORK! LEFT IN JUST TO AVOID DOING IT AGAIN...WILL CLEAN UP AT END?

# let's try some variables, as taken from an example here > https://github.com/galaxykate/tracery/issues/14

rules = {
    "name": ["Cheri","Fox","Morgana","Jedoo","Brick","Shadow","Krox","Urga","Zelph"],
"story": ["#name.capitalize# was a great #occupation#, and this song tells of #heroTheir# adventure. #name.capitalize# #didStuff#, then #heroThey# #didStuff#, then #heroThey# went home to read a book."] ,
"monster": ["dragon","ogre","witch","wizard","goblin","golem","giant","sphinx","warlord"] , 
"setPronouns": ["[heroThey:they][heroThem:them][heroTheir:their][heroTheirs:theirs]","[heroThey:she][heroThem:her][heroTheir:her][heroTheirs:hers]","[heroThey:he][heroThem:him][heroTheir:his][heroTheirs:his]"] , "setOccupation": ["[occupation:baker][didStuff:baked bread,decorated cupcakes,folded dough,made croissants,iced a cake]","[occupation:warrior][didStuff:fought #monster.a#,saved a village from #monster.a#,battled #monster.a#,defeated #monster.a#]"] ,
"origin": ["#[#setPronouns#][#setOccupation#][hero:#name#]story#"]
}
grammar = tracery.Grammar(rules) # create a grammar object from the rules
grammar.add_modifiers(base_english) # add pre-programmed modifiers

for i in range(3):
    print (grammar.flatten("#story#")) # and flatten, starting with origin rule

Fox was a great ((occupation)), and this song tells of ((heroTheir)) adventure. Jedoo ((didStuff)), then ((heroThey)) ((didStuff)), then ((heroThey)) went home to read a book.
Zelph was a great ((occupation)), and this song tells of ((heroTheir)) adventure. Cheri ((didStuff)), then ((heroThey)) ((didStuff)), then ((heroThey)) went home to read a book.
Morgana was a great ((occupation)), and this song tells of ((heroTheir)) adventure. Jedoo ((didStuff)), then ((heroThey)) ((didStuff)), then ((heroThey)) went home to read a book.


In [80]:
class person(id):
    id = id
    print("poo")

symbols = {
    "like": ["benches", "trees", "grass"]
}

x = person("X")

print(x)

grammar = tracery.Grammar(symbols) # create a grammar object from the rules
grammar.add_modifiers(base_english) # add pre-programmed modifiers

poo


TypeError: cannot create 'builtin_function_or_method' instances

In [97]:
# basic classes

class Dog:
    kind = 'canine'
    
    def __init__(self, name):
        self.name = name    

print(Dog.kind) # any flipping dog

r = Dog('Rex')
print(r.name, r.kind)
f = Dog('Fido')
print(f.name, f.kind)

canine
Rex canine
Fido canine


In [119]:
# so, let's try this with a town. A new grammar for each company!


# YES THIS IS RESTATING LOADS OF STUFF I KNOW

def loadData():
    with open('corpora/townprefixes.txt','r') as f:
        townprefixes = [line.strip() for line in f]
    with open('corpora/townsuffixes.txt','r') as f:
        townsuffixes = [line.strip() for line in f]
    with open('corpora/monopolystreets.txt','r') as f:
        street = [line.strip() for line in f]

    # print("prefixes = ",townprefixes)
    # print("suffixes =",townsuffixes)

rules = {
    "name": ["butcher","baker","candlestick maker"],
    "address": "#number# #street.capitalize#, #town.capitalize#, #zipcode#",
    "number": ["#d0#","#d0##d#","#d0##d##d#","#d0##d##d#","#d0##d##d#","#d0##d##d#","#d0##d##d##d#","#d0##d##d##d#","#d0##d##d##d#","#d0##d##d##d#","#d0##d##d##d#","#d0##d##d##d#","#d0##d##d##d#","#d0##d##d##d#","#d0##d##d##d#"],
    "street": street,
    "town": "#prefix.capitalize##suffix#",
    "prefix": townprefixes,
    "suffix": townsuffixes,
    "zipcode": "#d##d##d##d##d#",
    "d":["0","1","2","3","4","5","6","7","8","9"],
    "d0":["1","2","3","4","5","6","7","8","9"]
}

loadData()
grammar = tracery.Grammar(rules) # create a grammar object from the rules
grammar.add_modifiers(base_english) # add pre-programmed modifiers

class Business:
    kind = 'business'
    def __init__(self,name):
        self.name = grammar.flatten("#name.capitalize#")
        self.town = grammar.flatten("#town#")
        
        
print(Business.kind)

a = Business('Name')
print (a.kind, a.name, a.town)

business
business Butcher Kilochester


In [133]:
# ok, so that works...now let's define categories, and create a few new businesses per category


rules = {
    "cat": ["butchers","bakers","accountants","hitmen"], #to expand later
}

grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)


class Category:
    kind = 'category'
    def __init__(self,name):
        self.name = grammar.flatten("#cat.capitalize#")
        
a = Category('hm')
print(a.name)

Hitmen


# Mottoes

Companies in the old Yellow Pages have great mottoes, e.g.

    The {biggest/best/oldest/rarest/most sought-after} {business category/prices/selection} {in/near/{north/south/east/west/just outside} of} {town/county}{!/!!/!!!}

or 

    We put the _x_ in _y_ (where _y_ is a word associated with the business type)

or 

    Home of the {category} {doctor/specialists/repairers/practitioners}
    
or

    You've seen the rest, now find the best!
    
or

    Complete systems - one year warranty
    
or 

    {list of related services, with • separators}
    
or 

    From {one thing related to category} to {another thing related to category}
    
or 

    FREE ESTIMATES and NO PAYMENT UPFRONT
    INTEREST-FREE CREDIT

(perhaps a category of "benefits"?)

In [140]:
# the biggest selection in warburton!

loadData()
rules = {
    "motto": ["The #superlative# #thing# #located# #town#"],
    "superlative": ["biggest","best","oldest","rarest","most sought-after","nicest","prettiest"],
    "thing": ["prices","selection","collection","range","budget option"],
    "located": ["in","near","#dir# of"],
    "dir": ["north","south","east","west","just outside"],
    "town": "#prefix.capitalize##suffix#",
    "prefix": townprefixes,
    "suffix": townsuffixes
}

grammar = tracery.Grammar(rules) # create a grammar object from the rules
grammar.add_modifiers(base_english) # add pre-programmed modifiers

for i in range(30):
    print (grammar.flatten("#motto#")) # and flatten, starting with origin rule

The biggest prices near Quebecton
The nicest selection south of Foxtrotside
The best prices north of Romeoster
The rarest prices in Oscarview
The best prices in Limafield
The most sought-after collection in Limafield
The biggest prices in Zuluburn
The biggest selection near Deltabridge
The nicest selection east of Tangoland
The oldest selection in Limaton
The biggest budget option just outside of Xrayston
The most sought-after budget option near Limaport
The best budget option near Deltaford
The oldest selection in Deltadam
The oldest range near Kiloview
The biggest collection north of Whiskeyford
The biggest range east of Zuluton
The nicest budget option in Indiachester
The prettiest selection just outside of Xrayland
The prettiest range in Zuluville
The best selection near Sierrason
The oldest collection west of Deltaston
The biggest collection south of Golfford
The oldest selection just outside of Quebecstone
The nicest prices near Limaster
The oldest collection south of Kilobridge


In [161]:
# we put the x in y

with open('corpora/words_alpha.txt','r') as f:
    words = [line.strip() for line in f]

rules = {
    "category":words
}

grammar = tracery.Grammar(rules) # create a grammar object from the rules
grammar.add_modifiers(base_english) # add pre-programmed modifiers

for i in range(30):
    print (grammar.flatten("we put the #category# in #category#")) # and flatten, starting with origin rule

we put the latrine in hematozoon
we put the uvver in wheyisness
we put the bestrews in precited
we put the turpeths in rentree
we put the eben in meteorically
we put the hairs in deliver
we put the chalcanthite in deliriate
we put the cryptographers in eodiscid
we put the drogherman in withdraws
we put the agonises in heartlands
we put the miolithic in charadrii
we put the anthogenous in disbelief
we put the nonerosive in coiffed
we put the kotows in ruttishly
we put the explication in thymogenic
we put the subforeman in merd
we put the windages in armipotent
we put the backbearing in discrownment
we put the fablemaker in unhealably
we put the overstalely in backvelder
we put the pegboard in gogga
we put the peaked in uncontinually
we put the pantisocratical in untedded
we put the meteoroid in alumines
we put the defilements in ytterbia
we put the skimmia in subterposition
we put the lambly in atropine
we put the stoot in hazanim
we put the rouille in acetylide
we put the fruitworm in 